## Imports

In [1]:
import sklearn
sklearn.__file__


'/home/suzane/anaconda3/lib/python3.7/site-packages/sklearn/__init__.py'

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random
import math
import collections
from os import listdir
from os.path import isfile, join
from scipy.spatial.distance import cosine
import time
import pickle

import numpy as np
import umap
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn_extra.cluster import KMedoids
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.datasets.samples_generator import make_blobs

from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram

from pprint import pprint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

np.random.seed(42)
plt.rcParams['figure.figsize'] = [15, 5]

## Construção da Base de Dados

In [ ]:
# PEGA PATH DE TODAS IMAGENS E COLOCA EM UM ARRAY
all_paths = []
classes = listdir('./natural_images_100/')
all_classes = []

for root, dirs, files in os.walk('./natural_images_100/'):
    for d in dirs:
        for f in listdir(root + d):
            all_classes.append(d)
            all_paths.append(root + d + '/' + f)
# classes
# len(all_paths)

In [ ]:
# COLOCA TDS IMAGENS EM UM ARRAY DE IMAGENS PIL 224x224
all_images = np.empty(len(all_paths), dtype=object)

for n in range(0, len(all_paths)):
    all_images[n] = image.load_img(all_paths[n], target_size=(224, 224))

In [ ]:
def print_image(all_images_index):
    img = all_images[all_images_index]
    plt.imshow(img)
    plt.show()
    plt.close()
    
# print_image(0)

## Criando o modelo

In [ ]:
model = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

# model.summary()

In [ ]:
# calculating to all features
# SALVA ARRAY DE FEATURES EM UM GRANDE ARRAY PARA TDS FEATURES
all_features = np.empty((len(all_images), model._nested_outputs.shape[1]))
feature_list_np = []
feature_list = []

for i in range(len(all_images)):
    x = image.img_to_array(all_images[i])
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    feature_list_np.append(np.array(features))
    feature_list.append((np.array(features)).flatten())

    for feature in features:
        all_features[i] = feature

In [ ]:
print(np.shape(all_features))
print(np.shape(all_features[0]))
print()
print(np.shape(feature_list))
print(np.shape(feature_list[0]))

In [ ]:
# PICKLE

pickle_file_name = 'features.pkl'

#def dump_features(dump):
#    with open(pickle_file_name, 'w') as f:
#        pickle.dump(dump, f)

#def load_features():
#    with open(pickle_file_name, 'r') as f:
#        pickle.load(f)

d = {'path': all_paths, 'class': all_classes, 'features': feature_list}
df_features = pd.DataFrame(data=d)

df_features.to_pickle(pickle_file_name)

#unpickled_df = pd.read_pickle(pickle_file_name)

In [ ]:
all_features

In [ ]:
feature_list

In [ ]:
# printa features da primeira imagem
len(all_features)
pprint(all_features[0])
for a in all_features:
    print(a)

In [ ]:
# Verificando shape e tipo das features da prmeira imagem
print(all_features[0].shape, features.dtype, '\n')

# Mostrando as features.
pprint(features[0])

In [ ]:
# COMPARA DUAS IMAGENS PARECIDAS PRA VALIDAR AS FEATURES
index_img_1 = 0
index_img_2 = 5
distance = cosine(feature_list[index_img_1], feature_list[index_img_2])
print(distance)
print_image(index_img_1)
print_image(index_img_2)

# 2 Aplicação de Algoritmos de Agrupamento de Dados

### 2.1 Partitioning-based

In [ ]:
# Inicializa Dataframe para plotar resultados

images_transformed = umap.UMAP(n_neighbors=5).fit_transform(all_features.data)
df = pd.DataFrame(images_transformed, columns=['x', 'y'])
df.head()

### 2.1.a - Kmeans

In [ ]:
kmeans = KMeans(n_clusters=len(classes), random_state=0).fit(np.array(np.array(feature_list)))
print('Inércia do Kmeans: ', kmeans.inertia_)

In [ ]:
df_kmeans = df
df_kmeans['label'] = kmeans.labels_
df_kmeans.head()

In [ ]:
_ = sns.scatterplot(x='x', y='y', data=df_kmeans, hue='label', palette='rainbow', legend='full')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


### 2.1.b - KMedoids

In [ ]:
kmedoids = KMedoids(n_clusters=len(classes), random_state=0).fit(feature_list)
print('Inércia do KMedoids: ', kmedoids.inertia_)
print(kmedoids.labels_)
# reduced_data = PCA(n_components=len(classes)).fit_transform(feature_list)

# print(dir(reduced_data))

In [ ]:
len(kmedoids.labels_)

In [ ]:
df_kmedoids = df
df_kmedoids['label'] = kmedoids.labels_
df_kmedoids.head()


In [ ]:
_ = sns.scatterplot(x='x', y='y', data=df_kmedoids, hue='label', palette='rainbow', legend='full')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


### 2.1.c - Mini Batch KMeans

In [ ]:
miniBatchKmeans = MiniBatchKMeans(n_clusters=len(classes), random_state=0).fit(feature_list)

print('Inércia do Mini Batch Kmeans: {}\n'.format(miniBatchKmeans.inertia_))
print('Labels: ', miniBatchKmeans.labels_)


In [ ]:
df_miniBatchKmeans = df
df_miniBatchKmeans['label'] = miniBatchKmeans.labels_
df_miniBatchKmeans.head()


In [ ]:
_ = sns.scatterplot(x='x', y='y', data=df_miniBatchKmeans, hue='label', palette='rainbow', legend='full')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


### 2.2 Hierarchical-based

### 2.2.a Hierarchical Agglomerative Clustering - Single Linkage

In [ ]:
from scipy.cluster.hierarchy import ward, fcluster
from scipy.spatial.distance import pdist
Z = ward(pdist(feature_list))
clusters = fcluster(Z, t=0.9, criterion='distance')


In [ ]:
df_single = df
linkage_matrix_single = linkage(df_single, method='single')
plt.title('Hierarchical Clustering Dendrogram (Single)')
plt.xlabel('Sample index')
plt.ylabel('Distance')
d = dendrogram(linkage_matrix_single)


In [ ]:
labels = fcluster(linkage_matrix_single, 1.6, 'distance')
df_single['cluster_labels'] = labels
_ = sns.scatterplot(x='x', y='y', data=df_single, hue='cluster_labels', palette='tab10', legend=False)

### 2.2.b Hierarchical Agglomerative Clustering - Complete Linkage

In [ ]:
df_complete = df
linkage_matrix_complete = linkage(df_complete, method='complete')
plt.title('Hierarchical Clustering Dendrogram (Complete)')
plt.xlabel('Sample index')
plt.ylabel('Distance')
d = dendrogram(linkage_matrix_complete)

In [ ]:
df_complete['cluster_labels'] = labels
_ = sns.scatterplot(x='x', y='y', data=df_complete, hue='cluster_labels', palette='tab10', legend=False)

### 2.2.c Hierarchical Agglomerative Clustering - Average Linkage

In [ ]:
df_average = df
linkage_matrix = linkage(df_average, method='average')
d = dendrogram(linkage_matrix)

In [ ]:
labels = fcluster(linkage_matrix, 3.5, 'distance')
df_average['cluster_labels'] = labels
_ = sns.scatterplot(x='x', y='y', data=df_average, hue='cluster_labels', palette='tab10', legend=False)


### Density-based

In [ ]:
#  sklearn.cluster.DBSCAN

### Grid-based

### Model-based

# 3 Avaliação do resultado dos agrupamentos de dados

### Quantitativas

#### Interno

#### Externo

#### Relativo

### Subjetiva

In [ ]:

for i in range(len(classes)):
    print("Cluster", i, classes[i])
    for j in range(len(kmedoids.labels_)):
        if(kmedoids.labels_[j] == i):
            plt.rcParams["figure.figsize"] = (1,1)
            ax = plt.subplot(111)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            print_image(j)